# Spectral unmixing of fluorescent dyes

Adapted from *Marina Alterman, Yoav Schechner, and Aryeh Weiss, “Multiplexed Fluorescence Unmixing” (2010): 1–8.*


A mixture of $n_{dyes}$ dyes is excited with $S$ different laser line configurations, and the emitted flourescence is measured at $F$ diffrent wavelength bands. For a particular exitation source $s$ and filter band $f$ the corresponding intensity of the image $i_{f,s}$ is given by

$$i_{f,s} = \frac{q_{elect}^{cam}(f)}{g_{gray}^{cam}} \sum_{d=0}^{n_{dyes}} \mathcal{P}\left( N_{d} \left( f,s \right) \right) + \frac{1}{g_{gray}^{cam}} \mathcal{N}\left( 0,\sigma^2_{elect} \right)$$

where

$i_{f,s} \; \left[ \textrm{graylevel} \right]$ gray-scale value of signal in emission filter band $f$ with excitation source $s$.

$g_{gray}^{cam} \; \left[ \textrm{elect} \cdot \textrm{graylevel}^{-1} \right]$ conversion factor from photoelectrons to camera gray-scale values.

$q_{elect}^{cam}(f) \; \left[ \textrm{elect} \cdot \textrm{photon}^{-1} \right]$ quantum efficiency of the detector in filter band $f$.

$n_{dyes} \; \left[ \textrm{int} \right]$ number of dyes.

$N_{d} \left( f,s \right) \; \left[ \textrm{photons} \right]$ expected number of photons from dye $d$ in filter band $f$ with exication source $s$. $N_{0} \left( f,s \right)$ is the expected number of background photons.

$\sigma_{elect} \; \left[ \textrm{electron} \right]$ is the standard deviation of the sensor noise.

$\mathcal{P} \left( \lambda \right)$ is a poisson distribution with rate $\lambda$.

$\mathcal{N} \left( \mu, \sigma^2 \right)$ is a normal distribution with mean $\mu$ and variance $\sigma^2$.



The expected number of photons from dye $d$ in filter band $f$ with exication source $s$ can be determined from the total collected number of photons for that dye by

$$N_{d} \left( f,s \right) = \alpha_{d}(s) \epsilon_{d}(f) N_d^{Tot}$$

where

$N_d^{Tot}$ is the total number of photons collected from dye $d$.

$\alpha_{d}(s)$ is the normalised absorption of dye $d$ of source $s$. This factor can take into account different exciation powers for different sources $s$.

$\epsilon_{d}(f)$ is the normalised emission of dye $d$ in band $f$. This factor can take into account different transmission coefficicents for different bands $f$.

$ 0 \leq \alpha_{d}(s), \; 0 \leq \epsilon_{d}(f) \; \forall \; s, d$ and $\sum_{f,s}\alpha_{d}(s) \epsilon_{d}(f) = 1$.






### signal to noise
From *Sheppard, C., Min Gu, Maitreyee Roy, C. Sheppard, Min Gu, and Maitreyee Roy. “Signal‐to‐noise Ratio in Confocal Microscope Systems.” Journal of Microscopy (January 3, 1992).*

$$SNR = \frac{\eta N_p}{\sqrt{\eta N_p + N_n^2}}$$

Where $\eta$ is the quantum efficiency, $N_p$ is the number of signal photons per pixel and $N_n$ is the sensor noise in electrons per pixel.

Simulation - each fluorophore has an emission spectrum simulated by a center wavelength and a spectral half width.  
Represent all noise as shot-noise, so if N photons collected $\sqrt{N}$ noise.  
Minimize negative log-likehood function.  
Can add L1 like regularization term to cost function.

## setup

In [2]:
import matplotlib.pyplot as plt

In [3]:
%matplotlib inline

## image simulation

In [4]:
from unmixer import generate, infer, compare
from numpy.random import normal

# create coefficients and configuration
# could optionally load from JSON files (see test/resources for examples)
config = {'noise': [0.1, 0.1]}
coeff = {'emission': normal(size=(32,)), 'excitation': normal(size=(5,))}
conc = {'concentrations': 100}

signal, count = generate(coeff=coeff, config=config)
estimate = infer(signal=signal, coeff=coeff)
accuracy = compare(estimate, count)

print(accuracy)

42.2733853897


In [8]:
100*1e-9*6.022*1e23*1e3*1e-7*1e-7*1e-7

0.060219999999999996